In [2]:
from z3 import *
import numpy as np
import time

#------------------------------------------------------------------------------
# Utils
#------------------------------------------------------------------------------

def maximum(x):
    m = x[0]
    for v in x[1:]:
        m = If(v > m, v, m)
    return m

def precedes(a1, a2):
    if len(a1) == 1 and len(a2) == 1:
        return a1[0] > a2[0]
    return Or(a1[0] > a2[0], And(a1[0] == a2[0], precedes(a1[1:], a2[1:])))

#------------------------------------------------------------------------------
# Parameters
#------------------------------------------------------------------------------

def run_model_on_instance(file):
    with open(file) as f:
        m = int(next(f))
        n = int(next(f))
        l = [int(e) for e in next(f).split()]
        s = [int(e) for e in next(f).split()]
        D = np.genfromtxt(f, dtype=int).tolist()
    return m, n, l, s, D

#------------------------------------------------------------------------------
# Model
#------------------------------------------------------------------------------

def SMT(m, n, l, s, D):
    COURIERS = range(m)
    ITEMS = range(n)

    print(f"{m}\n{l}\n{n}\n{s}\n\n")

    #------------------------------------------------------------------------------
    # Variables
    #------------------------------------------------------------------------------

    # A = Function('A', IntSort(), IntSort(), IntSort())
    # O = Function('O', IntSort(), IntSort(), IntSort())
    # dist = Function('dist', IntSort(), IntSort())

    A = [ [ Int("a_%s_%s" % (i+1, j+1)) for j in ITEMS ]
        for i in COURIERS ]
    
    O = [ [ Int("o_%s_%s" % (i+1, j+1)) for j in ITEMS ]
        for i in COURIERS ]

    dist = [ Int("dist_%s" % (i+1) )for i in COURIERS ]

    # opt = Optimize()
    solver = Solver()
    start_time = time.time()

    #------------------------------------------------------------------------------
    # Constraints
    #------------------------------------------------------------------------------

    # Total items size less than total couriers capacity
    solver.add(Sum([l[i] for i in COURIERS]) >= Sum([s[j] for j in ITEMS]))

    # Constraints to create A
    for i in COURIERS:
        for j in ITEMS:
            solver.add(A[i][j] >= 0)
    for i in COURIERS:
        assignment = [A[i][j] for j in ITEMS]
        solver.add(And(Sum(assignment) > 0, Sum(assignment) <= l[i]))
    for j in ITEMS:
        assignment = [A[i][j] for i in COURIERS]
        solver.add(And(Sum(assignment) == s[j], maximum(assignment) == s[j]))

    # Lexicografic order constraint
    for i in range(m-1):
        assignment1 = [A[i][j] for j in ITEMS]
        assignment2 = [A[i+1][j] for j in ITEMS]
        sum1 = Sum(assignment1)
        sum2 = Sum(assignment2)
        condition = And(l[i] - sum1 >= 0, l[i+1] - sum2 >= 0, l[i+1] - sum1 >= 0, l[i] - sum2 >= 0)
        solver.add(Implies(condition, precedes(assignment1, assignment2)))

    encoding_A_time = time.time() - start_time
    print(f"Time spent encoding A: {encoding_A_time:3.3} seconds")

    # Constraints to create O
    for i in COURIERS:
        for j in ITEMS:
            solver.add(If(A[i][j] == 0, O[i][j] == 0, O[i][j] > 0))
    for i in COURIERS:
        order_items = [If(O[i][j] > 0, O[i][j], 0) for j in ITEMS]
        non_zero_items = [If(order_items[j] != 0, order_items[j], -j) for j in ITEMS]
        count = Sum([If(non_zero_items[j] > 0, 1, 0) for j in ITEMS])
        solver.add(Distinct(non_zero_items))
        solver.add(And([And(0 <= order_items[j], order_items[j] <= count) for j in ITEMS]))
        solver.add(Sum([If(order_items[e] == 1, 1, 0) for e in ITEMS]) == 1)

    encoding_O_time = time.time() - encoding_A_time
    print(f"Time spent encoding O: {encoding_O_time:3.3} seconds")

    # Constraint to create dist
    for i in COURIERS:
        order_items = [O[i][j] for j in ITEMS]
        c = Sum([If(order_items[j] > 0, 1, 0) for j in ITEMS])
        dist_expr = Sum([
            Sum([
                If(And(order_items[j1] != 0, order_items[j2] - order_items[j1] == 1), D[j1][j2], 0)
                for j2 in ITEMS
            ])
            for j1 in ITEMS
        ])
        dist_expr += Sum([If(order_items[j0] == 1, D[n][j0], 0) for j0 in ITEMS])
        dist_expr += Sum([If(order_items[jn] == c, D[jn][n], 0) for jn in ITEMS])
        solver.add(dist[i] == dist_expr)

    encoding_dist_time = time.time() - encoding_O_time
    print(f"Time spent encoding dist: {encoding_dist_time:3.3} seconds")

    #------------------------------------------------------------------------------
    # Objective
    #------------------------------------------------------------------------------

    obj = Int('obj')
    solver.add(obj == maximum([dist[i] for i in COURIERS]))

    #------------------------------------------------------------------------------
    # Search Strategy
    #------------------------------------------------------------------------------

    lower_bound = max([D[n][j] + D[j][n] for j in ITEMS])
    solver.add(obj >= lower_bound)

    # Solve the SMT problem
    # opt.minimize(obj)

    encoding_time = time.time() - start_time
    print(f"Starting search after: {encoding_time:3.3} seconds with lowerbound: [{lower_bound}]\n")
    solver.add(time.time() - start_time - encoding_time < 300000)  # Set timeout to 5 minutes

    if solver.check() != sat:
        print ("failed to solve")
    while solver.check() == sat:
        # if time.time() - start_time - encoding_time >= timeout:
        #     break
        model = solver.model()
        result_A = [ [ model.evaluate(A[i][j]) for j in ITEMS ] 
            for i in COURIERS ]
        result_O = [ [ model.evaluate(O[i][j]) for j in ITEMS ] 
            for i in COURIERS ]
        result_dist = [ model.evaluate(dist[i]) for i in COURIERS ]
        result_objective = model.evaluate(obj)
        for i in COURIERS:
            print_matrix(result_A[i])
        print()
        for i in COURIERS:
            print_matrix(result_O[i])
        print()
        print_matrix(result_dist)
        print(f"Intermediate objective value: {result_objective}\n")
        solver.add(obj < result_objective)
        
        # for i in COURIERS:
        #     for j in ITEMS:
        #         opt.add(Distinct(A[i][j], A[i][j]))

    print(f"\n\nFinal objective: {result_objective}")
    final_time = time.time() - start_time
    print(f"Finished in: {final_time:3.3} seconds\n")


SMT(*run_model_on_instance("./instances/inst02.dat"))

6
[190, 185, 185, 190, 195, 185]
9
[11, 11, 23, 16, 2, 1, 24, 14, 20]


Time spent encoding A: 0.025 seconds
Time spent encoding O: 1.69e+09 seconds
Time spent encoding dist: 0.201 seconds
Starting search after: 0.24 seconds with lowerbound: [226]

[11, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 11, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 23, 0, 0, 0, 24, 0, 0]
[0, 0, 0, 16, 0, 0, 0, 0, 20]
[0, 0, 0, 0, 2, 0, 0, 14, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]

[1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 2, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 2, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 2, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]

[174, 226, 116, 290, 303, 192]
Intermediate objective value: 303

[11, 0, 0, 16, 0, 0, 0, 0, 0]
[0, 11, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 23, 0, 2, 0, 24, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 14, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 20]

[1, 0, 0, 2, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 2, 0, 3, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0,